<a href="https://colab.research.google.com/github/jsebastianquiroga/PUJ_NLP_IA/blob/main/proyecto/motor_recomendaci%C3%B3n_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Procesamiento de Lenguaje Natural.</strong></h1>
<h2> Para: Ing. Juan Pablo Pajaro Hernandez, M.Sc.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Oscar Eduardo Correcha Guzman.</br></h2>
<h2>Jhon Alberto Aguirre Ardila.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>

**Este colab puede ser ejecutado por cualquier usuario sin necesidad de cargar los archivos al entorno, debido a que la conexión está configurada para acceder a los archivos desde la nube**.


<h1> <strong>Taller 1: Twitter User Gender Classification.</strong></h1>

En el presente Notebook, se exploraran los datos del data set de Kaggel: Twitter User Gender Classification. https://www.kaggle.com/crowdflower/twitter-user-gender-classification/

Este conjunto de datos, fue utilziado para entranar un modelo de clasificación de genero conocido como "CrowdFlower AI".

Cuyo proposito era inferir mendiante el perfil de un usuario de Twitter si el usuario era un hombre, mujer o compañia.

Se tienen un total de 20.000 observaciones, conformadas con un nombre de usuario, tuit aleatorio, perfil, la imagen de la cuenta, la ubicación.

<ol>
_unit_id: un id único para el usuario</ol>
<ol>_golden: si el usuario se incluyó en el patrón oro del modelo; TRUE o FALSE</ol>
<ol>_unit_state: estado de la observación; uno de finalized (para contributor-judged) o golden (para gold standard observations)</ol>
<ol>_trusted_judgments: número de juicios de confianza (int); siempre 3 para las observaciones no gold, y lo que puede ser un identificador único para las observaciones gold standard</ol>
<ol>_last_judgment_at: fecha y hora de la última resolución del contribuyente; en blanco para las observaciones gold standard</ol>
<ol>gender: uno de masculino, femenino o marca (para perfiles no humanos)</ol>
<ol>gender:confidence: valor flotante que representa la confianza en el sexo indicado</ol>
<ol>profile_yn: "no" aquí parece significar que el perfil debía formar parte del conjunto de datos pero no estaba disponible cuando los colaboradores fueron a juzgarlo</ol>
<ol>profile_yn:confidence: confianza en la existencia/no existencia del perfil</ol>
<ol>created: fecha y hora de creación del perfil</ol>
<ol>description: descripción del perfil del usuario</ol>
<ol>fav_number: número de tweets que el usuario ha marcado como favoritos</ol>
<ol>gender_gold: si el perfil es dorado, ¿cuál es el género?</ol>
<ol>link_color: el color del enlace en el perfil, como valor hexadecimal</ol>
<ol>name: nombre del usuario</ol>
<ol>profile_yn_gold: si el valor s/n del perfil es dorado</ol>
<ol>profileimage: un enlace a la imagen del perfil</ol>
<ol>retweet_count: número de veces que el usuario ha retuiteado (o posiblemente, ha sido retuiteado)</ol>
<ol>sidebar_color: color de la barra lateral del perfil, como valor hexadecimal</ol>
<ol>text: texto de un tweet aleatorio del usuario</ol>
<ol>tweet_coord: si el usuario tiene activada la localización, las coordenadas</ol>
<ol>como una cadena con el formato "[latitud, longitud]"</ol>
<ol>tweet_count: número de tweets que ha publicado el usuario</ol>
<ol>tweet_created: cuando se creó el tweet aleatorio (en la columna de texto)</ol>
<ol>tweet_id: el id del tweet aleatorio</ol>
<ol>tweet_location: ubicación del tweet; parece no estar especialmente normalizada</ol>
<ol>user_timezone: la zona horaria del usuario</ol>

El presente notebook, estará dividido en divido en las siguientes partes:
<ol>
<li> TBD:. </li>
<li> TBD. </li>
<li> TBD.</li>
</ol>

***

## <h1><strong> 1. Cargar el data set: </h1></strong>

El dataset del link en referencia, fue descargado previamente y subido a google drive, para poder cargar el presente notebook sin problemas al momento de cargar la información. </br>

In [1]:
import pandas as pd
# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth

from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

book_data = '17js_-seTF_YoCHxdMuGgcgKeypa7q9iX'
download = drive.CreateFile({'id': book_data})
download.GetContentFile('book_data.csv')


df_book_data = pd.read_csv("book_data.csv")

Books_rating = '11KpHbg3yI-XauV3pvWrIWzonk8uhc3ZB'
download = drive.CreateFile({'id': Books_rating})
download.GetContentFile('Books_rating.csv')


df_books_rating = pd.read_csv("Books_rating.csv")

print(df_book_data.columns)
print(df_books_rating.columns)

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')
Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')


In [2]:
# Fusionar los dos dataframes en la columna 'Title'
#merged_df = pd.merge(df_book_data, df_books_rating, on='Title', how='inner')
#merged_df = merged_df[['Title', 'description','authors','categories', 'review/text']]
#merged_df = merged_df.dropna()
merged_df = df_book_data[['Title', 'authors', 'description', 'categories']]
merged_df.shape

(212404, 4)

In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.3 MB/s eta 0:00:00


In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct 22 02:11:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Ensure you have GPU available
if not torch.cuda.is_available():
    raise ValueError("No GPU found, please ensure you're running on a GPU instance.")

# Use CUDA device
device = torch.device("cuda")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# If you have multiple GPUs, wrap the model with DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    # Tokenize on CPU
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    # Move tokens to GPU
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Assuming merged_df is loaded or defined elsewhere
# Generate embeddings
for column in ['Title', 'authors', 'description', 'categories']:#, 'review/text']:
    merged_df[f'{column}_embedding'] = merged_df[column].apply(generate_embeddings)

# Concatenate all embeddings
merged_df['combined_embedding'] = merged_df.apply(lambda row: np.concatenate([row[f'{column}_embedding'] for column in ['Title', 'authors', 'description', 'categories', 'review/text']]), axis=1)

# Train the k-NN model
knn_model = NearestNeighbors(n_neighbors=6)  # Fetching 6 neighbors including the book itself
knn_model.fit(np.stack(merged_df['combined_embedding'].values))

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Exclude the first index as it will be the book itself
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example of usage
recommend_books('I want to read something fantasy and love')

ValueError: ignored

In [ ]:
# Leer los embeddings y el modelo
merged_df = pd.read_pickle('merged_df_embeddings.pkl')
knn_model = joblib.load('knn_model.pkl')

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Excluir el primer índice ya que será el libro mismo
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Ejemplo de uso
recommend_books('I want to read something fantasy and love')


from transformers import BertTokenizer, BertModel

# Cargar el tokenizer y el modelo de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Generar embeddings
for column in ['Title', 'authors', 'description', 'categories', 'review/text']:
    merged_df[f'{column}_embedding'] = merged_df[column].apply(generate_embeddings)

from sklearn.neighbors import NearestNeighbors

# Concatenar todos los embeddings
merged_df['combined_embedding'] = merged_df.apply(lambda row: np.concatenate([row[f'{column}_embedding'] for column in ['Title', 'authors', 'description', 'categories', 'review/text']]), axis=1)

# Ajustar el modelo k-NN
knn_model = NearestNeighbors(n_neighbors=5)  # Buscar los 5 libros más cercanos, más el libro mismo
knn_model.fit(np.stack(merged_df['combined_embedding'].values))

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Excluir el primer índice ya que será el libro mismo
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Ejemplo de uso
recommend_books('I want to read something fantasy and love')